# Markov Chain from scratch

Last update: Sept 22nd, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Example

Consider a worker who, at any given time $ t $, is either unemployed (state 0) or employed (state 1).

Suppose that, over a one month period,

1. An unemployed worker finds a job with probability $ \alpha \in (0, 1) $.  
1. An employed worker loses her job and becomes unemployed with probability $ \beta \in (0, 1) $.  


In terms of a Markov model, we have

- $ S = \{ 0, 1\} $  
- $ P(0, 1) = \alpha $ and $ P(1, 0) = \beta $  


We can write out the transition probabilities in matrix form as


$$
P
= \left(
\begin{array}{cc}
    1 - \alpha & \alpha \\
    \beta & 1 - \beta
\end{array}
  \right)
$$

We assume $\alpha = 0.5$ and $\beta = 0.1$ as an example.

In [2]:
# An example of Markov matrix construction
alpha = 0.5
beta = 0.1


In [3]:
# Construct P
P = np.array([[1 - alpha, alpha], [beta, 1 - beta]])

In [6]:
# Or, fill in the numbers
P = np.zeros((2, 2))
P[0, 0] = 1 - alpha
P[0, 1] = alpha
P[1, 0] = beta
P[1, 1] = 1 - beta

## Chain Simulation, single agent

How do we randomly pick a state that we are going to jump into?
1. Generate a uniformly distributed random number U(0, 1)
2. Suppose we are at state 0. If this number is greater than $1-\alpha$ then we are going to jump to state 1. Otherwise, we stay at state 0.

Then we iterate forward for T times.

In [18]:
# One step iteration

cur_state = 0
cum_prob = np.cumsum(P[cur_state, :])
tmp = 0.4
next_state = np.searchsorted(cum_prob, tmp)

In [12]:
P[0, :]

array([0.5, 0.5])

In [16]:
cum_prob

array([0.5, 1. ])

In [19]:
next_state

0

In [13]:
# Iterate T times
T = 1000

ini_state = 0 # Starting from state 0 for example
res = np.zeros(T, dtype = int) # Initialize the result record
res[0] = ini_state
prob_draw = np.random.rand(T) # Draw all the probabilities in a row
for i in range(T - 1):
    cum_prob = np.cumsum(P[res[i], :])
    res[i + 1] = np.searchsorted(cum_prob, prob_draw[i])


In [52]:
# Wrap into a function

def Markov_simulation_single(P, ini_state, T):
    res = np.zeros(T, dtype = int) # Initialize the result record
    res[0] = ini_state
    prob_draw = np.random.rand(T) # Draw all the probabilities in a row
    for i in range(T - 1):
        cum_prob = np.cumsum(P[res[i], :])
        res[i + 1] = np.searchsorted(cum_prob, prob_draw[i])
        
    return res

simulation_res = Markov_simulation_single(P, 0, 1000)


In [50]:
# A bit more optimization
def Markov_simulation_single_new(P, ini_state, T):
    res = np.zeros(T, dtype = int) # Initialize the result record
    res[0] = ini_state
    prob_draw = np.random.rand(T) # Draw all the probabilities in a row
    cum_prob = np.cumsum(P, axis = 1)
    for i in range(T - 1):
        res[i + 1] = np.searchsorted(cum_prob[res[i], :], prob_draw[i])
        
    return res

simulation_res = Markov_simulation_single_new(P, 0, 1000)

## Chain simulation, distribution

The distribution is characterized by a vector. Then the simulation is just vector-matrix multiplication

In [77]:
def Markov_simulation_dist(P, ini_dist, T):
    n = len(ini_dist)
    res = np.zeros((n, T))
    res[:, 0] = ini_dist.transpose()
    for i in range(T - 1):
        res[:, i + 1] = P.transpose() @ res[:, i]
        
    return res

In [78]:
ini_dist = np.array([0.5, 0.5])
res = Markov_simulation_dist(P, ini_dist, 100)